# Imports and Settings 

In [1]:
import numpy as np
import pandas as pd
from pulp import *

# Data Wrangling

In [2]:
df = pd.read_json('../../data/json/players_stats.json', orient='records', encoding='utf-8')
print(df.shape)
df.head(10)

(18055, 59)


,Acceleration,Aggression,Agility,Balance,BallControl,Body Type,Composure,Crossing,Curve,Dislike,...,Work Rate,age,birth_date,height,name,photo_url,player_hashtags,player_traits,positions,weight
0,92,48,90,95,96,Messi,97,77,90,197,...,Medium / Medium,30,1987/Jun/24,170,Lionel Messi,https://cdn.sofifa.org/1x/18/players/158023.png,"#Dribbler,#FK Specialist,#Acrobat,#Clinical Fi...","Finesse Shot,Long Shot Taker,Speed Dribbler,Pl...","CF,ST,RW",72
1,89,63,89,63,93,C. Ronaldo,95,85,81,260,...,High / Low,32,1985/Feb/5,187,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.org/1x/18/players/20801.png,"#Speedster,#Dribbler,#Distance Shooter,#Acroba...","Power Free Kick,Diver,Flair,Long Shot Taker,Sp...","LW,ST",83
2,94,56,96,82,95,Neymar,92,75,82,74,...,High / Medium,25,1992/Feb/5,175,Neymar da Silva Santos Jr.,https://cdn.sofifa.org/1x/18/players/190871.png,"#Speedster,#Dribbler,#Acrobat","Diver,Flair,Speed Dribbler,Technical Dribbler,...",LW,68
3,88,87,86,78,91,Normal,85,77,86,48,...,High / Medium,30,1987/Jan/24,182,Luis Suárez,https://cdn.sofifa.org/1x/18/players/176580.png,"#Dribbler,#Acrobat,#Clinical Finisher","Diver,Tries To Beat Defensive Line,Outside Foo...",ST,86
4,56,29,51,35,48,Normal,70,15,14,11,...,Medium / Medium,31,1986/Mar/27,193,Manuel Neuer,https://cdn.sofifa.org/1x/18/players/167495.png,,"GK Long Throw,GK 1,on,1 rush,GK Rushes Out of ...",GK,92
5,57,38,60,43,42,Lean,64,17,21,15,...,Medium / Medium,26,1990/Nov/7,193,David De Gea Quintana,https://cdn.sofifa.org/1x/18/players/193080.png,,"GK Long Throw,GK Saves With Feet",GK,76
6,78,73,82,79,90,Normal,86,92,85,17,...,High / High,26,1991/Jun/28,181,Kevin De Bruyne,https://cdn.sofifa.org/1x/18/players/192985.png,"#Dribbler,#Playmaker ,#Engine,#Distance Shoot...","Leadership,Early Crosser,Long Passer,Long Shot...","CAM,CM",68
7,79,80,78,80,89,Normal,87,62,77,16,...,High / Medium,28,1988/Aug/21,185,Robert Lewandowski,https://cdn.sofifa.org/1x/18/players/188545.png,#Clinical Finisher,"Injury Free,Finesse Shot,Chip Shot",ST,79
8,93,54,93,93,93,Normal,89,81,83,36,...,High / Medium,26,1991/Jan/7,173,Eden Hazard,https://cdn.sofifa.org/1x/18/players/183277.png,"#Speedster,#Dribbler,#Acrobat","Tries To Beat Defensive Line,Finesse Shot,Flai...","LW,CF",76
9,65,60,71,71,89,Normal,85,85,85,18,...,Medium / Medium,27,1990/Jan/4,183,Toni Kroos,https://cdn.sofifa.org/1x/18/players/182521.png,"#Playmaker ,#Distance Shooter","Long Passer,Long Shot Taker,Playmaker,Technica...","CM,CDM",76


In [3]:
df.isnull().any()

Acceleration                False
Aggression                  False
Agility                     False
Balance                     False
BallControl                 False
Body Type                   False
Composure                   False
Crossing                    False
Curve                       False
Dislike                     False
Dribbling                   False
FKAccuracy                  False
Finishing                   False
Follow                      False
GKDiving                    False
GKHandling                  False
GKKicking                   False
GKPositioning               False
GKReflexes                  False
HeadingAccuracy             False
Interceptions               False
International Reputation    False
Jumping                     False
Like                        False
LongPassing                 False
LongShots                   False
Marking                     False
Overall Rating              False
Penalties                   False
Position      

In [4]:
print(np.unique(df['Preferred Foot']))
print(np.unique(df['Body Type']))

['Left' 'Right']
['Akinfenwa' 'C. Ronaldo' 'Courtois' 'Lean' 'Messi' 'Neymar' 'Normal'
 'Shaqiri' 'Stocky']


### Data Cleaning

In [5]:
df.age = df.age.astype(np.int)

### Prefered Foot Conversion

In [6]:
df['Preferred Foot'] = df['Preferred Foot'].map({'Left': 0, 'Right': 1})

### Price Conversion 

In [7]:
currencies = np.array([list(value)[0] for value in df.Value.tolist()])
print(np.unique(currencies, return_counts=True))

(array(['+', '-', '5', '6', '7', '8', '€'], dtype='<U1'), array([    7,    14,     3,    68,    53,     2, 17908]))


In [8]:
def value_to_num(col):
    if pd.isnull(col): return 0
    
    value = col.replace('€', '').replace('M', '').replace('K', '')
    
    if col[-1] == 'M': unit = 1e6
    elif col[-1] == 'K': unit = 1e3
    else: unit = 1
    
    return float(value)*unit

df.Value = df.Value.apply(value_to_num)
df.Wage = df.Wage.apply(value_to_num)
df['Release Clause'] = df['Release Clause'].apply(value_to_num)

In [9]:
df[['name', 'Value', 'Wage', 'Release Clause']].head()

,name,Value,Wage,Release Clause
0,Lionel Messi,118500000.0,565000.0,242900000.0
1,C. Ronaldo dos Santos Aveiro,95500000.0,565000.0,195800000.0
2,Neymar da Silva Santos Jr.,119500000.0,280000.0,230000000.0
3,Luis Suárez,97000000.0,510000.0,198900000.0
4,Manuel Neuer,61000000.0,230000.0,100700000.0


In [10]:
df.to_csv('../../data/csv/players_stats.csv', index=False)

# Optimization 

<img src='images/fifa_positions.png' width='600'>

In [11]:
def get_optimized_team(df, max_budget, n_gk=0, 
                       n_cb=0, n_lb=0, n_rb=0, n_lwb=0, n_rwb=0, 
                       n_cdm=0, n_cm=0, n_rm=0, n_lm=0, n_cam=0, 
                       n_rw=0, n_lw=0, n_cf=0, n_st=0):

    list_dicts = [  {},   {},   {},   {},    {},    {},    {},   {},   {},   {},    {},   {},   {},   {},   {}]
    list_pos   = ['GK', 'CB', 'LB', 'RB', 'LWB', 'RWB', 'CDM', 'CM', 'LM', 'RM', 'CAM', 'CF', 'LW', 'RW', 'ST']
    list_n     = [n_gk, n_cb, n_lb, n_rb, n_lwb, n_rwb, n_cdm, n_cm, n_lm, n_rm, n_cam, n_cf, n_lw, n_rw, n_st]

    players_ids = [str(i) for i in range(df.shape[0])]
    potentials = {i:potential for i, potential in zip(players_ids, df.Potential.values)}
    values = {i:value for i, value in zip(players_ids, df.Value.values)}
        
    for player_dict, player_pos, n_players in zip(list_dicts, list_pos, list_n):
        if n_players <= 0: continue
            
        for i, pos in zip(players_ids, df.positions.values):
            player_dict.update({i:1} if pos.find(player_pos) >= 0 else {i:0})
            
    players_vars = LpVariable.dicts(name="Players", indexs=players_ids, cat=LpBinary)

    prob = LpProblem(name="SoFIFA", sense=LpMaximize)
    
    # objective function
    prob += lpSum([potentials[i]*players_vars[i] for i in players_ids])
        
    # constraints
    prob += lpSum([players_vars[i] for i in players_ids]) == sum(list_n)
    prob += lpSum([values[i]*players_vars[i] for i in players_ids]) <= max_budget
    for dict_player, n_players in zip(list_dicts, list_n):
        if n_players > 0:
            prob += lpSum([dict_player[i]*players_vars[i] for i in players_ids]) == n_players    

    prob.solve()
    
    idxs = np.array([int(v.name.split("_")[-1]) for v in prob.variables() if v.varValue == 1])
    mask_players = np.zeros(df.shape[0], dtype=np.bool)
    mask_players[idxs] = True
    
    return prob.status, mask_players

In [12]:
status, mask_players = get_optimized_team(df, 1000000, n_gk=1, n_cb=2, n_lwb=1, n_rwb=1, n_cdm=1, n_cm=1, n_cam=1, n_cf=2, n_st=1)
print(LpStatus[status], mask_players.sum())

Optimal 11


In [13]:
df.iloc[mask_players][['positions', 'name', 'age', 'Overall Rating', 'Potential', 'Value']]

,positions,name,age,Overall Rating,Potential,Value
69,"LM,RM",Yannick Carrasco,23,85,90,0.0
175,"CAM,CM",Oscar dos Santos Emboaba,25,83,85,0.0
197,"CB,CDM",Javier Mascherano,33,83,83,0.0
485,"ST,CF",Marcus Berg,30,80,80,0.0
1810,"RM,LM",Derlis González,23,75,81,0.0
6093,"RW,CF",Dennis Man,18,69,83,0.0
7440,GK,José Contreras,22,68,78,0.0
8164,"RB,LB",Reeve Frosler,19,67,81,0.0
10000,"RB,CB",Cristian Manea,19,66,78,0.0
13788,"RWB,RB,RM",Dujon Sterling,17,62,82,525000.0
